In [ ]:
library("gdalcubes")
library("rstac")
library("tibble")
library("sf")
library("geojsonsf")

In [ ]:
s = stac("https://planetarycomputer.microsoft.com/api/stac/v1/")

In [ ]:
b <- c(-2009488, -715776, 1401061, 2597757)
bounds <- matrix(c(b[1],b[2],
                   b[1],b[4],
                   b[3],b[4],
                   b[3],b[2],
                   b[1],b[2]
                ),ncol=2, byrow=TRUE)

resolution <- 1000
epsg <- 32198
tt<-st_sfc(st_polygon(list(bounds)))
st_crs(tt) = epsg
bounds_ll <- tt |>
  st_transform(crs = 4326) |>
  sfc_geojson() |>
  jsonlite::fromJSON()
bounds_ll

In [ ]:
it_obj <- s |>
    stac_search(intersects=bounds_ll, collections=c('io-lulc'),limit=5000) |> post_request()
it_obj <- it_obj |> items_sign(sign_fn = sign_planetary_computer())
st<-stac_image_collection(it_obj$features,asset_names=c("data"))
st
#

In [ ]:
v = cube_view(srs = paste0("EPSG:",epsg),  extent = list(t0 = "2020-06-01", t1 = "2020-06-01",
              left = b[1], right = b[3],  top = b[4], bottom = b[2]),
              dx = resolution, dy = resolution, dt = "P1Y",aggregation = "first", resampling = "mode")

gdalcubes_options(threads = 4)
mosaic = raster_cube(st, v, chunking = c(1, 5000, 5000)) |>
    select_bands(c("data"))
    #reduce_time(c("max(data)"))



In [ ]:
mosaic |> 
    write_tif(
        dir = "/home/jovyan/",
        prefix = "io-lulc",
        COG = TRUE
    )
